# Create a Table from LLM Output


Before downloading a model, you have to provide an access key to Huggingface.
Go to ... and create a key. Copy the key into a textfile that your application can load.

In [2]:
import os
jp = os.path.join
import sys
import datetime
import json
T_now = datetime.datetime.now
from openai import OpenAI



In [3]:
openai_api_key = open(jp(os.path.expanduser("~"), ".secrets", "openai_pmolnar_gsu_edu_msa8700.apikey"), "r").read().strip()
os.environ["OPENAI_API_KEY"] = openai_api_key
client = OpenAI(api_key = openai_api_key)

## Create Helper Functions

In [5]:
def llm(input_text, max_new_tokens: int = 1000, openai_client = client):
    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=max_new_tokens,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": input_text, ## f"Answer the following question:\n\n{question}\n"
            }
        ]
    )
    return completion.choices[0].message.content.strip()

## Try out

In [6]:
txt = """
Make a list of 17 tourist attractions in Paris, order them by their popularity with the most popular first.
For each attaction state the Name, Part of the city, short description, and number of annual visitors.
Format the output as PSV with fields "Name" | "Location" | "Description" | "Visitors".
"""
T_0 = T_now()
data = llm(txt, 10000)
print(f"Elapsed time: {T_now()-T_0}")
print(data[:2000]) ### limit output ... sometime there's clutter at the end...

Elapsed time: 0:00:10.394650
Here’s a list of 17 popular tourist attractions in Paris, formatted as requested:

```
Name | Location | Description | Visitors
Eiffel Tower | Champ de Mars, 5 Avenue Anatole France | Iconic wrought-iron lattice tower, symbol of Paris. | 7 million
Louvre Museum | Rue de Rivoli | World’s largest art museum, home to the Mona Lisa. | 9.6 million
Notre-Dame Cathedral | Île de la Cité | Famous Gothic cathedral known for its stunning architecture. | 12 million
Sacré-Cœur Basilica | 35 Rue du Chevalier de la Barre | Basilica located at the highest point in the city, offers panoramic views. | 10 million
Champs-Élysées | Avenue des Champs-Élysées | Famous avenue known for shopping, theaters, and cafés. | 14 million
Arc de Triomphe | Place Charles de Gaulle | Monument honoring those who fought for France, with a viewing platform. | 1.5 million
Montmartre | 18th Arrondissement | Historic district known for its bohemian past and artists. | 10 million
Palace of Versaill

## Process LLM output to structured data

You can use LLMs to produce structured data like this table of popular attractions in Paris. However, LLMs may not always produce perfectly formatted output text. Some text processing to cleanup the output might be required.

In [7]:
import pandas as pd
from io import StringIO

Let's split the output text into lines, and then split each line by the "|" (pipe symbol) 

In [8]:
raw_dat = [ list(map(lambda s: str(s).strip(), line.split('|'))) for line in data.split('\n') ]
print(raw_dat[:20])

[['Here’s a list of 17 popular tourist attractions in Paris, formatted as requested:'], [''], ['```'], ['Name', 'Location', 'Description', 'Visitors'], ['Eiffel Tower', 'Champ de Mars, 5 Avenue Anatole France', 'Iconic wrought-iron lattice tower, symbol of Paris.', '7 million'], ['Louvre Museum', 'Rue de Rivoli', 'World’s largest art museum, home to the Mona Lisa.', '9.6 million'], ['Notre-Dame Cathedral', 'Île de la Cité', 'Famous Gothic cathedral known for its stunning architecture.', '12 million'], ['Sacré-Cœur Basilica', '35 Rue du Chevalier de la Barre', 'Basilica located at the highest point in the city, offers panoramic views.', '10 million'], ['Champs-Élysées', 'Avenue des Champs-Élysées', 'Famous avenue known for shopping, theaters, and cafés.', '14 million'], ['Arc de Triomphe', 'Place Charles de Gaulle', 'Monument honoring those who fought for France, with a viewing platform.', '1.5 million'], ['Montmartre', '18th Arrondissement', 'Historic district known for its bohemian pa

As you can see there are some blank lines. We can filter those out and then use Pandas to convert the text into a DataFrame

In [9]:
# raw_dat = [ list(map(lambda s: str(s).strip(), line.split('|'))) for line in data.split('\n') ]
filt_dat = list(filter(lambda lst: len(lst)==4, raw_dat))
filt_dat[:5]

[['Name', 'Location', 'Description', 'Visitors'],
 ['Eiffel Tower',
  'Champ de Mars, 5 Avenue Anatole France',
  'Iconic wrought-iron lattice tower, symbol of Paris.',
  '7 million'],
 ['Louvre Museum',
  'Rue de Rivoli',
  'World’s largest art museum, home to the Mona Lisa.',
  '9.6 million'],
 ['Notre-Dame Cathedral',
  'Île de la Cité',
  'Famous Gothic cathedral known for its stunning architecture.',
  '12 million'],
 ['Sacré-Cœur Basilica',
  '35 Rue du Chevalier de la Barre',
  'Basilica located at the highest point in the city, offers panoramic views.',
  '10 million']]

In [10]:
df = pd.DataFrame(filt_dat)
df.columns = ["Name", "Location", "Description", "Visitors"]
print(f"Number of rows: {df.shape[0]:,}")
display(df)

Number of rows: 18


,Name,Location,Description,Visitors
0,Name,Location,Description,Visitors
1,Eiffel Tower,"Champ de Mars, 5 Avenue Anatole France","Iconic wrought-iron lattice tower, symbol of P...",7 million
2,Louvre Museum,Rue de Rivoli,"World’s largest art museum, home to the Mona L...",9.6 million
3,Notre-Dame Cathedral,Île de la Cité,Famous Gothic cathedral known for its stunning...,12 million
4,Sacré-Cœur Basilica,35 Rue du Chevalier de la Barre,Basilica located at the highest point in the c...,10 million
5,Champs-Élysées,Avenue des Champs-Élysées,"Famous avenue known for shopping, theaters, an...",14 million
6,Arc de Triomphe,Place Charles de Gaulle,"Monument honoring those who fought for France,...",1.5 million
7,Montmartre,18th Arrondissement,Historic district known for its bohemian past ...,10 million
8,Palace of Versailles,"Place d'Armes, Versailles",Former royal residence known for its opulent a...,10 million
9,Musée d'Orsay,1 Rue de la Légion d'Honneur,"Museum housed in a Beaux-Arts railway station,...",3.5 million


## Travel guide function

In [11]:
def travel_guide(city: str, num_attractions: int = 17) -> pd.DataFrame:
    txt = f"""
Make a list of {num_attractions} tourist attractions in {city}, order them by their popularity with the most popular first.
For each attaction state the Name, Part of the city, short description, and number of annual visitors.
Format the output as PSV with fields "Name" | "Location" | "Description" | "Visitors".
"""
    data = llm(txt, 10000)
    raw_dat = [ list(map(lambda s: str(s).strip(), line.split('|'))) for line in data.split('\n') ]
    filt_dat = list(filter(lambda lst: len(lst)==4, raw_dat))
    if filt_dat[0][0].lower() == "name":
        filt_dat = filt_dat[1:]
    df = pd.DataFrame(filt_dat, columns=["Name", "Location", "Description", "Visitors"])
    print(f"Number of rows: {df.shape[0]:,}")
    return df

In [12]:
berlin_df = travel_guide("Berlin", 5)
display(berlin_df)

Number of rows: 5


,Name,Location,Description,Visitors
0,Brandenburg Gate,Mitte,An iconic neoclassical monument that symbolize...,14 million
1,Berlin Wall Memorial,Mitte,A preserved section of the Berlin Wall that in...,1.5 million
2,Reichstag Building,Mitte,"The seat of the German Parliament, known for i...",3 million
3,Museum Island,Mitte,A UNESCO World Heritage site that houses five ...,3.5 million
4,Checkpoint Charlie,Kreuzberg,The famous crossing point between East and Wes...,1 million


In [13]:
barcelona_df = travel_guide("Barcelona", 5)
display(barcelona_df)

Number of rows: 5


,Name,Location,Description,Visitors
0,Sagrada Família,Eixample,"An iconic basilica designed by Antoni Gaudí, k...",4.5 million
1,Park Güell,Gràcia,"A public park also designed by Gaudí, featurin...",3 million
2,La Rambla,Ciutat Vella,"A vibrant street in the heart of the city, fam...",10 million
3,Casa Batlló,Eixample,"A modernist building designed by Gaudí, celebr...",1 million
4,Gothic Quarter,Ciutat Vella,"The historic center of Barcelona, known for it...",7 million


In [14]:
atlanta_df = travel_guide("Atlanta", 5)
display(atlanta_df)

Number of rows: 5


,Name,Location,Description,Visitors
0,Georgia Aquarium,Downtown Atlanta,The Georgia Aquarium is one of the largest aqu...,2 million
1,World of Coca-Cola,Downtown Atlanta,This museum showcases the history of the Coca-...,1.2 million
2,Atlanta Botanical Garden,Midtown Atlanta,A beautiful garden featuring a variety of plan...,1 million
3,Martin Luther King Jr. National Historical Park,Sweet Auburn,This park includes the childhood home of Marti...,"800,000"
4,Fox Theatre,Midtown Atlanta,A historic performing arts venue known for its...,"600,000"
